# Nigep example using MNIST dataset

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src/nigep/*'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.nigep import NigepOOD
from keras.datasets import cifar10, cifar100
import keras.applications as applications
from keras.models import Sequential
import keras.layers as layers
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, EarlyStopping


### Load and preprocess MNIST dataset from Keras package

In [2]:
# Train Data
(x_train, y_train), (_, _) = cifar10.load_data()

x_train = x_train.astype('float32')
x_train = x_train / 255
y_train = to_categorical(y_train, 10)

# OOD
(_, _), (x_test_ood, y_test_ood) = cifar100.load_data()

x_test_ood = x_test_ood.astype('float32')

x_test_ood = x_test_ood / 255

y_test = to_categorical(y_test_ood, 100)

### Build and compile the MLP model

In [3]:
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(32,32,3))

model = Sequential(layers=[
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(10, activation='softmax')
])

model.summary()

model.compile(optimizer=SGD(learning_rate = 0.001, momentum = 0.9), loss='categorical_crossentropy', metrics = ['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 10)                5130      
                                                                 
Total params: 14719818 (56.15 MB)
Trainable params: 14719818 (56.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Execute Nigep to obtain the Generalization Profile results

In [ ]:
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, min_delta=0.001, patience=5, verbose=1)
erl_stopping = EarlyStopping(patience=4, monitor='val_loss', verbose=1)
callbacks = [lr_reduce, erl_stopping]


nigep = NigepOOD(
    execution_name='cifar10',
    x_data=x_train,
    y_data=y_train,
    ood_x_data=x_test_ood,
    ood_y_data=y_test_ood,
    model=model,
    batch_size=64,
    input_shape=(32,32),
    class_mode='categorical',
    target_names=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
    epochs=20,
    callbacks=callbacks
)

nigep.execute()

Training with Noise: 0
Training with Noise: 0.1


In [ ]:
nigep.plot_and_save_generalization_profile()